In [ ]:
import os
import sys
import polars as pl
import polars.selectors as cs

from datetime import datetime

PROJECT_PATH = os.path.abspath(os.path.join("..", ".."))
PIPELINE_PATH = os.path.join(PROJECT_PATH, "pipeline")
DATA_DIRECTORY = os.path.join(PROJECT_PATH, "data")

if PIPELINE_PATH not in sys.path:
    sys.path.append(PIPELINE_PATH)
    
from utils.paths import PROCESSED_PICARRO_DATA_DIRECTORY, POSTPROCESSED_DATA_DIRECTORY
from utils.import_data import import_acropolis_system_data
from utils.plot_dataframes import plot_column_difference, plot_column

assert(os.path.exists(DATA_DIRECTORY))
assert(os.path.exists(POSTPROCESSED_DATA_DIRECTORY))

In [ ]:
# functions
def join_picarro_data(df: pl.DataFrame, df_p: pl.DataFrame) -> pl.DataFrame:
    return df.sort("datetime") \
    .join(df_p, on="datetime") \
    .group_by_dynamic("datetime", every='1h', group_by=["system_id", "system_name"]) \
    .agg([
            cs.numeric().mean(),
            pl.col("gmp343_corrected").std().alias("gmp343_corrected_std"),
            pl.col("gmp343_corrected").var().alias("gmp343_corrected_var")
        ]) 
            
def calculate_performance_metrics(df: pl.DataFrame, col:str = "gmp343_corrected") -> pl.DataFrame:
    return df.with_columns(diff = pl.col("CO2") - pl.col(col)) \
        .filter(pl.col("diff").is_not_nan()) \
        .select([
            (pl.col("diff").mean()).alias("mean_full_deployment"),
            (pl.col("diff").pow(2).mean()).alias("mse_full_deployment"),
            (pl.col("diff").pow(2).mean().sqrt()).alias("rmse_full_deployment")
        ])

In [ ]:
df_p_icos = pl.scan_parquet(os.path.join(PROCESSED_PICARRO_DATA_DIRECTORY,"Calibrated_1_min_ICOS_Picarro_G2401_529.parquet"))

start_date = datetime(2025, 2, 17, 15, 0, 0)
end_date = datetime(2025, 2 , 24, 7, 0, 0)

df_p_icos = df_p_icos.rename({"creation_timestamp": "datetime", "picarro_corrected": "CO2"}) \
    .select(["datetime", "CO2"]) \
    .with_columns(pl.col("datetime").cast(pl.Datetime("ms"))) \
    .filter(pl.col("datetime").is_between(start_date, end_date)) \
    .with_columns(system_name = pl.lit("Picarro_ICOS")) \
    .with_columns(system_id = pl.lit(529)) \
    .with_columns(pl.col("system_id").cast(pl.Int64)) \
    .collect()

In [ ]:
df_3 = import_acropolis_system_data(years = [2024,2025], 
                                    target_directory=POSTPROCESSED_DATA_DIRECTORY, 
                                    id=3, 
                                    prefix="1min").collect().pipe(join_picarro_data, df_p_icos)
df_6 = import_acropolis_system_data(years = [2024,2025], 
                                    target_directory=POSTPROCESSED_DATA_DIRECTORY, 
                                    id=6, 
                                    prefix="1min").collect().pipe(join_picarro_data, df_p_icos)

In [ ]:
df_3.with_columns(pl.col("datetime").cast(pl.Datetime("ms"))) \
    .filter(pl.col("datetime").is_between(start_date, end_date)) \
    .select("datetime", "CO2", "gmp343_corrected", "gmp343_corrected_std", "gmp343_corrected_var")

In [ ]:
print(calculate_performance_metrics(df_3, col="gmp343_corrected"))
print(calculate_performance_metrics(df_3, col="gmp343_edge_corrected"))

In [ ]:
print(calculate_performance_metrics(df_6, col="gmp343_corrected"))
print(calculate_performance_metrics(df_6, col="gmp343_edge_corrected"))

In [ ]:
plot_column_difference(df_3, datetime_col="datetime", col1="gmp343_corrected", col2="CO2", sample_size=100_000)
plot_column_difference(df_3, datetime_col="datetime", col1="gmp343_edge_corrected", col2="CO2", sample_size=100_000)
plot_column_difference(df_3, datetime_col="datetime", col1="gmp343_corrected", col2="gmp343_edge_corrected", sample_size=100_000)

In [ ]:
plot_column_difference(df_6, datetime_col="datetime", col1="gmp343_corrected", col2="CO2", sample_size=100_000)
plot_column_difference(df_6, datetime_col="datetime", col1="gmp343_edge_corrected", col2="CO2", sample_size=100_000)
plot_column_difference(df_6, datetime_col="datetime", col1="gmp343_corrected", col2="gmp343_edge_corrected", sample_size=100_000)

In [ ]:
plot_column(df_6, datetime_col="datetime", col1="gmp343_corrected", sample_size=100_000)